In [249]:
import os
import math
import tarfile
from six.moves import urllib
import hashlib
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TreebankWordTokenizer
from bs4 import BeautifulSoup
import xml.etree.ElementTree
import mailbox   
import lxml.html
import lxml.html.clean
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.stem import PorterStemmer
import email
from sklearn.pipeline import Pipeline
# Import the email modules we'll need
from email.parser import Parser
import mailparser as mp
from email.mime.text import MIMEText


#Giving filepaths for the data
dat_path = "datasets/spamham"
spam_path = dat_path + '/easy_spam'
ham_path = dat_path + '/easy_ham'
ar_dict = [(spam_path, 4)] #, {ham_path,2551}] 501
words_mails = []

for sp, num in ar_dict:
    tr_n = math.ceil(0.8 * int(num))
    print(tr_n)
    
    for file_num in range(tr_n):
        #Reading an email and getting the content
        data_mail = spam_PATH + '/mail_' + '97'#str(file_num)
        print(data_mail)
        
        fp = open(data_mail, 'rb')
        
        mail = mp.parse_from_file(data_mail)
        
        #msg = email.parser.BytesParser(policy=email.policy.default).parse(fp)
       # simplest = msg.get_body()
        
        #headers = email.parser.Parser(policy=email.policy.default).parsestr(str(msg))
        print(mail.body)
        
        #print(simplest)
        #print('To: {}'.format(headers['to']))
        #print('From: {}'.format(headers['from']))
        #print('Subject: {}'.format(headers['subject']))
        #print(msg)
        #print(msg.keys())
        
        #num_urls = counting_urls(content)
        #pd_in = preparing_data()
        #l = pd_in.transform(content)
        #words_mails.append(l)
        #print(len(words_mails))
            

                
class preparing_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, x, y=None):
        return self
    def transform(self, x, y=None):
        x.lower()
        x = cleaning_htmlent(x)
        
        content_words =  TreebankWordTokenizer().tokenize(content)
        ps = PorterStemmer()
        f_c =   [ps.stem(w) for w in content_words]
        
        return f_c
        


def cleaning_htmlent(text):
    doc = lxml.html.fromstring(text)
    cleaner = lxml.html.clean.Cleaner(style=True)
    doc = cleaner.clean_html(doc)
    content = doc.text_content()
    return content

def counting_urls(text):
#     soup = BeautifulSoup(text, 'html.parser')
#     num = soup.find_all('a')
#     print('number is ', (num))
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    num_urls = len(urls)
    return num_urls
  



4
datasets/spamham/easy_spam/mail_97
This is a multipart MIME message.

--= Multipart Boundary 0925021429
Content-Type: text/plain;
	charset="ISO-8859-1"
Content-Transfer-Encoding: 8bit

<html>

<head>
</head>

<body>

<p align="center"><span style="font-size:7.5pt;font-family:Arial;color:black">You
are receiving this email because you opted-in to receive special offers from
OptinDeals through one of our marketing partners. If you feel you have received
this email in error or do not wish to receive additional special offers, please
reply to this email with the word &quot;remove&quot; in the subject line or
follow the unsubscribe instructions below.<o:p>
</o:p>
</span></p>
<p><a href="http://www.inkjetrus.com/customer!.html"><img border="0" 
src="http://www.inkjetrus.com/clip%20art/toper.jpg" width="800" height="100"></a>
</p>
<div align="center">
  <strong><font face="Arial" size="6">Incredible Deals on Inkjet Cartridges!
</font></strong>
</div>
<div align="center">
  <strong><font fac

In [211]:
name = 'anahdfad'
str(name, 'utf-8')

TypeError: decoding str is not supported